# 🦷 Gogobe - PDF Magazine Scanner

Extract dental products from PDF magazines automatically!

## 📋 What this does:
1. Upload PDF magazine
2. Extract text
3. Find products & prices
4. Export to CSV/SQL
5. Load into Gogobe database

## 🚀 Run in Google Colab:
1. Go to: https://colab.research.google.com
2. File → Upload notebook
3. Upload this file
4. Run cells!


In [ ]:
# Install required libraries
!pip install -q PyPDF2 pdfplumber pandas openpyxl
print("✅ Libraries installed!")


In [ ]:
# Step 2: Upload PDF file
from google.colab import files
import io

print("📤 Upload your PDF magazine:")
uploaded = files.upload()

# Get the uploaded file
pdf_filename = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {pdf_filename}")
print(f"   Size: {len(uploaded[pdf_filename]) / 1024:.1f} KB")


In [ ]:
# Step 3: Extract text from PDF
import pdfplumber
import pandas as pd

def extract_text_from_pdf(pdf_path):
    """Extract text from all pages"""
    pages_text = []
    
    with pdfplumber.open(pdf_path) as pdf:
        print(f"📄 PDF has {len(pdf.pages)} pages\n")
        
        for i, page in enumerate(pdf.pages, 1):
            text = page.extract_text()
            if text:
                pages_text.append({
                    'page': i,
                    'text': text
                })
                print(f"   Page {i:3d}: {len(text):6d} characters")
    
    return pages_text

# Extract
print("🔍 Extracting text...\n")
pages = extract_text_from_pdf(pdf_filename)
print(f"\n✅ Extracted {len(pages)} pages")


In [ ]:
# Step 4: Find products with prices
import re

def find_products_with_prices(text, page_num):
    """Find products with prices in text"""
    products = []
    
    # Price patterns for different currencies
    patterns = [
        r'£([\d,]+(?:\.\d{2})?)',  # GBP: £1,234.56
        r'\$([\d,]+(?:\.\d{2})?)',  # USD: $1,234.56
        r'€([\d,]+(?:\.\d{2})?)',   # EUR: €1,234.56
        r'₪([\d,]+(?:\.\d{2})?)',   # ILS: ₪1,234.56
    ]
    
    currencies = ['GBP', 'USD', 'EUR', 'ILS']
    
    for pattern, currency in zip(patterns, currencies):
        for match in re.finditer(pattern, text):
            price_str = match.group(1).replace(',', '')
            try:
                price = float(price_str)
            except:
                continue
            
            if price < 5:  # Skip too small prices (likely page numbers)
                continue
            
            # Get context around price (300 chars before and after)
            start = max(0, match.start() - 300)
            end = min(len(text), match.end() + 300)
            context = text[start:end]
            
            # Try to find product name (lines before price)
            before = context[:match.start()-start]
            lines = [l.strip() for l in before.split('\n') if l.strip()]
            
            product_name = ''
            for line in reversed(lines):
                if len(line) > 10 and not re.match(r'^[£$€₪\d,\.\s]+$', line):
                    product_name = line
                    break
            
            if product_name and price > 0:
                products.append({
                    'page': page_num,
                    'name': product_name[:300],
                    'price': price,
                    'currency': currency,
                    'context': context[:500]
                })
    
    return products

# Search all pages
print("🔍 Searching for products with prices...\n")
all_products = []

for page_data in pages:
    products = find_products_with_prices(page_data['text'], page_data['page'])
    all_products.extend(products)
    if products:
        print(f"   Page {page_data['page']:3d}: {len(products)} products found")

print(f"\n✅ Found {len(all_products)} potential products")


In [ ]:
# Step 5: Clean and organize data
def is_valid_product(name):
    """Filter out non-products"""
    name_lower = name.lower()
    
    # Exclude keywords
    exclude = [
        'subscription', 'magazine', 'event', 'training',
        'course', 'seminar', 'conference', 'membership',
        'advertisement', 'sponsored', 'page', 'issue',
        'editorial', 'contents', 'classified'
    ]
    
    return not any(kw in name_lower for kw in exclude)

# Convert to DataFrame
df = pd.DataFrame(all_products)

if not df.empty:
    print("🧹 Cleaning data...\n")
    
    initial_count = len(df)
    
    # Remove duplicates
    df = df.drop_duplicates(subset=['name', 'price', 'currency'])
    print(f"   Removed {initial_count - len(df)} duplicates")
    
    # Filter by name length
    df = df[df['name'].str.len() > 15]
    
    # Filter by price (reasonable range)
    df = df[df['price'] > 10]
    df = df[df['price'] < 100000]
    
    # Filter valid products
    df = df[df['name'].apply(is_valid_product)]
    
    # Sort by price
    df = df.sort_values('price', ascending=False)
    df = df.reset_index(drop=True)
    
    print(f"   Final products: {len(df)}")
    print(f"\n✅ Data cleaned and ready!")
else:
    print("❌ No products found in PDF")


In [ ]:
# Step 6: View summary and preview
if not df.empty:
    print("=" * 60)
    print("📊 SUMMARY")
    print("=" * 60)
    print(f"Total products: {len(df)}")
    print(f"\nBy currency:")
    for curr in df['currency'].unique():
        count = len(df[df['currency'] == curr])
        total = df[df['currency'] == curr]['price'].sum()
        avg = df[df['currency'] == curr]['price'].mean()
        print(f"   {curr}: {count} products, Total: {curr} {total:,.2f}, Avg: {curr} {avg:,.2f}")
    
    print(f"\nPrice range: {df['price'].min():.2f} - {df['price'].max():.2f}")
    print(f"Average price: {df['price'].mean():.2f}")
    print("=" * 60)
    
    print("\n🔍 Preview (top 10 most expensive):\n")
    preview = df[['page', 'name', 'price', 'currency']].head(10)
    print(preview.to_string(index=False))



In [ ]:
# Step 7: Export to CSV
if not df.empty:
    # Prepare export data
    df_export = df[['page', 'name', 'price', 'currency']].copy()
    
    # Generate filename
    import os
    base_name = os.path.splitext(pdf_filename)[0]
    csv_filename = f"{base_name}_products.csv"
    
    # Save CSV
    df_export.to_csv(csv_filename, index=False, encoding='utf-8')
    
    print(f"\n✅ Saved to: {csv_filename}")
    print(f"   {len(df_export)} products exported")
    
    # Download file
    print("\n📥 Downloading CSV...")
    files.download(csv_filename)
    print("✅ Download complete!")


In [ ]:
# Step 8: Generate SQL for database
if not df.empty:
    sql_lines = [
        "-- Products extracted from PDF magazine",
        "-- Run this in pgAdmin on database 'gogobe'",
        "-- Or use: psql -U postgres -d gogobe -f thisfile.sql\n",
        "DO $$",
        "DECLARE",
        "    dental_id INTEGER;",
        "    cat_id INTEGER;",
        "    supp_id INTEGER;",
        "    pid BIGINT;",
        "BEGIN",
        "    -- Get vertical ID",
        "    SELECT id INTO dental_id FROM verticals WHERE slug = 'dental';",
        "    ",
        "    -- Get category ID (using first available)",
        "    SELECT id INTO cat_id FROM categories WHERE vertical_id = dental_id LIMIT 1;",
        "    ",
        "    -- Create a generic supplier for magazine extracts",
        "    INSERT INTO suppliers (name, slug, country)",
        "    VALUES ('PDF Magazine Extract', 'pdf-magazine', 'UK')",
        "    ON CONFLICT DO NOTHING;",
        "    ",
        "    SELECT id INTO supp_id FROM suppliers WHERE slug = 'pdf-magazine';",
        "    \n"
    ]
    
    # Add each product
    for idx, row in df.iterrows():
        name = row['name'].replace("'", "''")  # Escape quotes
        price = row['price']
        currency = row['currency']
        page = row['page']
        
        sql_lines.append(f"    -- Product {idx+1} (Page {page}): {name[:50]}...")
        sql_lines.append(f"    INSERT INTO products (name, vertical_id, category_id, description)")
        sql_lines.append(f"    VALUES (")
        sql_lines.append(f"        '{name}',")
        sql_lines.append(f"        dental_id,")
        sql_lines.append(f"        cat_id,")
        sql_lines.append(f"        'Extracted from {pdf_filename}, page {page}'")
        sql_lines.append(f"    )")
        sql_lines.append(f"    ON CONFLICT DO NOTHING")
        sql_lines.append(f"    RETURNING id INTO pid;")
        sql_lines.append(f"    ")
        sql_lines.append(f"    IF pid IS NOT NULL THEN")
        sql_lines.append(f"        INSERT INTO prices (product_id, supplier_id, price, currency, scraped_at)")
        sql_lines.append(f"        VALUES (pid, supp_id, {price}, '{currency}', NOW());")
        sql_lines.append(f"        RAISE NOTICE '✅ Product {idx+1}: {name[:40]}...';")
        sql_lines.append(f"    END IF;")
        sql_lines.append(f"    ")
    
    sql_lines.append("END $$;")
    sql_lines.append("")
    sql_lines.append("-- View results")
    sql_lines.append("SELECT p.name, pr.price, pr.currency, s.name as supplier")
    sql_lines.append("FROM products p")
    sql_lines.append("JOIN prices pr ON p.id = pr.product_id")
    sql_lines.append("JOIN suppliers s ON pr.supplier_id = s.id")
    sql_lines.append("WHERE p.description LIKE '%Extracted from%'")
    sql_lines.append("ORDER BY pr.price DESC;")
    
    # Save SQL file
    sql_filename = f"{base_name}_products.sql"
    with open(sql_filename, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sql_lines))
    
    print(f"\n✅ SQL file created: {sql_filename}")
    print(f"   {len(df)} INSERT statements generated")
    
    # Download SQL
    print("\n📥 Downloading SQL...")
    files.download(sql_filename)
    print("✅ Download complete!")


In [ ]:
# Step 9: Next Steps
print("\n" + "="*60)
print("🎉 PDF SCANNING COMPLETE!")
print("="*60)
print("\n📦 Files downloaded:")
print(f"   1. {csv_filename} - Spreadsheet format")
print(f"   2. {sql_filename} - Database ready")

print("\n🚀 Next steps:")
print("\n1️⃣  Open CSV in Excel/Google Sheets")
print("   - Review products")
print("   - Edit if needed")
print("   - Analyze data")

print("\n2️⃣  Load into PostgreSQL database:")
print("   On your computer, run:")
print(f'   psql -U postgres -d gogobe -f "{sql_filename}"')

print("\n3️⃣  Or use pgAdmin:")
print("   - Open pgAdmin")
print("   - Select database 'gogobe'")
print("   - Tools → Query Tool")
print(f"   - Open {sql_filename}")
print("   - Press F5 to run")

print("\n📊 Statistics:")
if not df.empty:
    print(f"   Products extracted: {len(df)}")
    print(f"   Total value: {df['price'].sum():,.2f}")
    print(f"   Pages scanned: {df['page'].nunique()}")
    print(f"   Success rate: {len(df)/len(all_products)*100:.1f}%")

print("\n💡 Tips for next time:")
print("   - Better PDFs = better results")
print("   - Clear formatting helps")
print("   - Review and clean data manually")
print("   - Build product categories")

print("\n🔄 Want to scan another PDF?")
print("   Just run all cells again!")
print("="*60)
